# Object Detection

In notebook [1_explore.ipynb](1_explore.ipynb) we saw how to use a pre-trained object detection model to identify objects in a static image. In this notebook, we consider which parts of the code we used are crucial in making a prediction, and remove the code which was merely used to explore the data. 

You might have noticed that this folder we are working in contains afew files we haven't talked about. This 'project' has been set up for us by our Super Star Application Developer, and contains everything we need to easily go from experiment to application. 

The process we will be using to create our application is called source-to-image, or s2i. Don't worry if you've never heard of it! We will step you through what you need to do! For now, take a look at the way in which the project is organised:

### Project Organization
```
.
├── README.md
├── LICENSE
├── requirements.txt        <- Used to install packages for s2i application
├── 1_explore.ipynb         <- Notebook for data and use case exploration
├── 2_predict.ipynb         <- Notebook for creating a predict function
├── 3_run_flask.ipynb       <- Notebook for running flask locally to test
├── 4_test_flask.ipynb      <- Notebook for testing flask requests
├── .gitignore              <- standard python gitignore
├── .s2i                    <- hidden folder for advanced s2i configuration
│   └── environment         <- s2i environment settings
├── gunicorn_config.py      <- configuration for gunicorn when run in OpenShift
├── prediction.py           <- the predict function called from Flask
└── wsgi.py                 <- basic Flask application
```

## Install Dependencies

In the previous notebook, all of our libraries were already installed for us because we selected the 'tensorflow' notebook image from the Jupyterhub spawner. However, when we run the model as part of an application we need to ensure we are packaging the model with the correct requirements. Take a look at the `requirements.txt` file to see which libraries we need for our application.


We can install these libraries into our environment:

In [ ]:
import sys
!pip install -r requirements.txt

## Create a Predict Function

Extract the prediction logic into a standalone python file, `prediction.py` in a `predict` function.  Also, make sure `requirements.txt` is updated with any additional packages you've used and need for prediction.

In [ ]:
import tensorflow as tf
import base64

model_dir = 'models/openimages_v4_ssd_mobilenet_v2_1'
saved_model = tf.saved_model.load(model_dir)
detector = saved_model.signatures['default']


def predict(body):
    base64img = body.get('image')
    img_bytes = base64.decodebytes(base64img.encode())
    detections = detect(img_bytes)
    cleaned = clean_detections(detections)
    
    return { 'detections': cleaned }


def detect(img):    
    image = tf.image.decode_jpeg(img, channels=3)
    converted_img  = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)
    num_detections = len(result["detection_scores"])
    
    output_dict = {key:value.numpy().tolist() for key, value in result.items()}
    output_dict['num_detections'] = num_detections
    
    return output_dict


def clean_detections(detections):
    cleaned = []
    # max_boxes = 10
    # num_detections = min(detections['num_detections'], max_boxes)
    num_detections = detections['num_detections']

    for i in range(0, num_detections):
        label = detections['detection_class_entities'][i].decode('utf-8')
        score = detections['detection_scores'][i]
        if score > 0.3:
            d = {
                'box': {
                    'yMin': detections['detection_boxes'][i][0],
                    'xMin': detections['detection_boxes'][i][1],
                    'yMax': detections['detection_boxes'][i][2],
                    'xMax': detections['detection_boxes'][i][3]
                },
                'class': label,
                'label': label,
                'score': score,
            }
            cleaned.append(d)

    return cleaned


def clean_detections(detections):
    cleaned = []
    max_boxes = 10
    num_detections = min(detections['num_detections'], max_boxes)

    for i in range(0, num_detections):
        d = {
            'box': {
                'yMin': detections['detection_boxes'][i][0],
                'xMin': detections['detection_boxes'][i][1],
                'yMax': detections['detection_boxes'][i][2],
                'xMax': detections['detection_boxes'][i][3]
            },
            'class': detections['detection_class_entities'][i].decode('utf-8'),
            'label': detections['detection_class_entities'][i].decode('utf-8'),
            'score': detections['detection_scores'][i],
        }
        cleaned.append(d)

    return cleaned

In [ ]:
import json

with open('sample-requests/twodogs.json') as json_file:
    data = json.load(json_file)
    
result = predict(data)
print(result['detections'])

So, you can see that we only need a few lines of code to be able to make a prediction - much of the code in the first notebook was for exploration only. 


We've put the code from this notebook into the `prediction.py` file, as that is where the source-to-image builder will look for our prediction function. 